# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments             Customer  \
0  Oct 31 2022  3:10PM  249739        19  ADV80007207  AdvaGen Pharma, Ltd   
1  Oct 31 2022  3:10PM  249739        19  ADV80007208  AdvaGen Pharma, Ltd   
2  Oct 31 2022  3:10PM  249739        19  ADV80007209  AdvaGen Pharma, Ltd   
3  Oct 31 2022  3:09PM  249738        10   0086133950    ISDIN Corporation   
4  Oct 31 2022  3:09PM  249738        10   0086133961    ISDIN Corporation   
5  Oct 31 2022  3:09PM  249738        10   0086133960    ISDIN Corporation   
6  Oct 31 2022  3:09PM  249738        10   0086133965    ISDIN Corporation   
7  Oct 31 2022  3:09PM  249738        10   0086133966    ISDIN Corporation   
8  Oct 31 2022  3:09PM  249738        10   0086133971    ISDIN Corporation   
9  Oct 31 2022  3:09PM  249738        10   0086133980    ISDIN Corporation   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
22  249730       Released          1
23  249733       Released          9
24  249735       Released          3
25  249738       Released         22
26  249739       Released          3

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
249730                NaN        NaN       1.0
249733                NaN        NaN       9.0
249735                NaN        NaN       3.0
249738                NaN        NaN      22.0
249739                NaN        NaN       3.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
249680                0.0        0.0       1.0
249684                0.0        3.0       8.0
249685                0.0       31.0      34.0
249693                0.0        0.0       1.0
249710                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
249680                  0          0         1
249684                  0          3         8
249685                  0         31        34
249693                  0          0         1
249710                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               249680          0          0         1
1               249684          0          3         8
2               249685          0         31        34
3               249693          0          0         1
4               249710          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               249680                            1
1               249684                   3        8
2               249685                  31       34
3               249693                            1
4               249710                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                         Customer
0   Oct 31 2022  3:10PM  249739        19              AdvaGen Pharma, Ltd
3   Oct 31 2022  3:09PM  249738        10                ISDIN Corporation
25  Oct 31 2022  2:54PM  249735        10                ISDIN Corporation
28  Oct 31 2022  2:42PM  249733        10                ISDIN Corporation
37  Oct 31 2022  2:04PM  249729        10                          Bio-PRF
41  Oct 31 2022  2:03PM  249730        16                  SHL Pharma, LLC
42  Oct 31 2022  1:56PM  249727        10                   ClearSpec, LLC
43  Oct 31 2022  1:32PM  249725        19           NAPP Technologies, LLC
44  Oct 31 2022  1:17PM  249722        20                   Exact-Rx, Inc.
67  Oct 31 2022  1:06PM  249685        15  Brookfield Pharmaceuticals, LLC

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                         Customer  \
0  Oct 31 2022  3:10PM  249739        19              AdvaGen Pharma, Ltd   
1  Oct 31 2022  3:09PM  249738        10                ISDIN Corporation   
2  Oct 31 2022  2:54PM  249735        10                ISDIN Corporation   
3  Oct 31 2022  2:42PM  249733        10                ISDIN Corporation   
4  Oct 31 2022  2:04PM  249729        10                          Bio-PRF   
5  Oct 31 2022  2:03PM  249730        16                  SHL Pharma, LLC   
6  Oct 31 2022  1:56PM  249727        10                   ClearSpec, LLC   
7  Oct 31 2022  1:32PM  249725        19           NAPP Technologies, LLC   
8  Oct 31 2022  1:17PM  249722        20                   Exact-Rx, Inc.   
9  Oct 31 2022  1:06PM  249685        15  Brookfield Pharmaceuticals, LLC   

  Completed Executing Released  
0                            3  
1                           22  
2                            3  
3                            9  
4                   3        1  
5                            1  
6                            1  
7                            1  
8                  12       11  
9                  31       34

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                         Customer  \
0  Oct 31 2022  3:10PM  249739        19              AdvaGen Pharma, Ltd   
1  Oct 31 2022  3:09PM  249738        10                ISDIN Corporation   
2  Oct 31 2022  2:54PM  249735        10                ISDIN Corporation   
3  Oct 31 2022  2:42PM  249733        10                ISDIN Corporation   
4  Oct 31 2022  2:04PM  249729        10                          Bio-PRF   
5  Oct 31 2022  2:03PM  249730        16                  SHL Pharma, LLC   
6  Oct 31 2022  1:56PM  249727        10                   ClearSpec, LLC   
7  Oct 31 2022  1:32PM  249725        19           NAPP Technologies, LLC   
8  Oct 31 2022  1:17PM  249722        20                   Exact-Rx, Inc.   
9  Oct 31 2022  1:06PM  249685        15  Brookfield Pharmaceuticals, LLC   

  Released Executing Completed  
0        3                      
1       22                      
2        3                      
3        9                      
4        1         3            
5        1                      
6        1                      
7        1                      
8       11        12            
9       34        31

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse             Customer Released  \
0  Oct 31 2022  3:10PM  249739        19  AdvaGen Pharma, Ltd        3   
1  Oct 31 2022  3:09PM  249738        10    ISDIN Corporation       22   
2  Oct 31 2022  2:54PM  249735        10    ISDIN Corporation        3   
3  Oct 31 2022  2:42PM  249733        10    ISDIN Corporation        9   
4  Oct 31 2022  2:04PM  249729        10              Bio-PRF        1   

  Executing Completed  
0                      
1                      
2                      
3                      
4         3

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse             Customer  Released  \
0  Oct 31 2022  3:10PM  249739        19  AdvaGen Pharma, Ltd       3.0   
1  Oct 31 2022  3:09PM  249738        10    ISDIN Corporation      22.0   
2  Oct 31 2022  2:54PM  249735        10    ISDIN Corporation       3.0   
3  Oct 31 2022  2:42PM  249733        10    ISDIN Corporation       9.0   
4  Oct 31 2022  2:04PM  249729        10              Bio-PRF       1.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        3.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse             Customer  Released  \
0  Oct 31 2022  3:10PM  249739        19  AdvaGen Pharma, Ltd       3.0   
1  Oct 31 2022  3:09PM  249738        10    ISDIN Corporation      22.0   
2  Oct 31 2022  2:54PM  249735        10    ISDIN Corporation       3.0   
3  Oct 31 2022  2:42PM  249733        10    ISDIN Corporation       9.0   
4  Oct 31 2022  2:04PM  249729        10              Bio-PRF       1.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        3.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1997816      40.0       12.0        0.0
15          499369      42.0       34.0        0.0
16          749139       2.0        1.0        2.0
19          499464       4.0        0.0        0.0
20          499402      12.0       12.0        0.0
21          998846       4.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1997816      40.0       12.0        0.0
1        15   499369      42.0       34.0        0.0
2        16   749139       2.0        1.0        2.0
3        19   499464       4.0        0.0        0.0
4        20   499402      12.0       12.0        0.0
5        21   998846       4.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      40.0       12.0        0.0
1        15      42.0       34.0        0.0
2        16       2.0        1.0        2.0
3        19       4.0        0.0        0.0
4        20      12.0       12.0        0.0
5        21       4.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   40.0
1        15  Released   42.0
2        16  Released    2.0
3        19  Released    4.0
4        20  Released   12.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15   16   19    20   21
Status                                    
Completed   0.0   0.0  2.0  0.0   0.0  0.0
Executing  12.0  34.0  1.0  0.0  12.0  0.0
Released   40.0  42.0  2.0  4.0  12.0  4.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15   16   19    20   21
0          Completed   0.0   0.0  2.0  0.0   0.0  0.0
1          Executing  12.0  34.0  1.0  0.0  12.0  0.0
2           Released  40.0  42.0  2.0  4.0  12.0  4.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15   16   19    20   21
0  Completed   0.0   0.0  2.0  0.0   0.0  0.0
1  Executing  12.0  34.0  1.0  0.0  12.0  0.0
2   Released  40.0  42.0  2.0  4.0  12.0  4.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()